# Modeling Old Homes

Now that we have explored our data, and cleaned it up into an acceptable format, we are ready to run some modeling techniques on it. We will begin by building multiple generic models on the data set to see how well it performs. After this, we will build a Neural Network to see if we can improve the performance from the baseline models.

In [1]:
#Import Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

### Loading Dataset

This dataset is already cleaned up and preprocessed. It is expected that this dataset has the following columns:
* SALEPRICE

> NOTE/TODO: Currently, we don't do anything with the PROPERTYZIP. We plan to build a model for each zipcode, but that is pending our preliminary results (to see if we need to even attempt such a thing).

In [2]:
# Loading Data
data = pd.read_csv('Old_Dataset.csv')

# Renaming columns to not include brackets, spaces, or commas
column_mapping = {}
for col in list(data.columns):

    new_col = col.replace(']', ')')
    new_col = new_col.replace('[', '(')
    new_col = new_col.replace(', ', '-')
    new_col = new_col.replace(' ', '')
    column_mapping[col] = new_col

data = data.rename(columns=column_mapping)

# Drop Nulls
data = data.dropna()

In [3]:
data.head()

,Unnamed:0,STYLE,ROOF,EXTFINISH,BASEMENT,ZIPCODE,HOOD,YEARSOLD,PREVSALEPRICE,PREVSALEDATE,SALEDATE,COUNTS,CDU,GRADE,CONDITION,LOTAREA,COUNTYTOTAL,FAIRMARKETTOTAL,LOCALTOTAL,COUNTYBUILDING,FAIRMARKETBUILDING,LOCALBUILDING,COUNTYLAND,FAIRMARKETLAND,LOCALLAND,STORIES,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,FINISHEDLIVINGAREA,SALEPRICE
0,641,-0.511977,-0.262557,-0.578541,-0.219149,-0.47813,0.304581,1.496347,-0.519136,0.010992,0.454293,2.024899,1.862384,2.179999,-0.018026,-0.308089,-0.881868,-0.992637,-0.993102,-0.753596,-0.892857,-0.892857,-1.001830,-0.998507,-1.001830,0.744710,5.670035,8.149506,0.679835,-1.013469,2.139410,68000.0
1,1437,1.388084,-0.262557,2.315631,-0.219149,-0.47813,0.405550,-1.397087,0.312065,-0.061015,0.742901,0.822061,0.139318,0.300263,-0.018026,-0.283961,0.043691,0.082386,0.082740,0.085562,0.133338,0.133338,-0.093901,-0.094567,-0.093901,0.744710,-0.406335,-0.156837,0.679835,0.720036,-0.265927,167000.0
2,1439,1.388084,-0.262557,-0.165088,-0.219149,-0.47813,0.405550,-1.397087,1.047818,-0.709378,0.407730,1.167430,0.139318,-0.326315,-0.018026,-0.283961,0.225418,0.262291,0.262783,0.310218,0.355571,0.355571,-0.093901,-0.094567,-0.093901,0.744710,0.808939,-0.156837,0.679835,0.720036,0.227340,210000.0
3,1470,0.046864,-0.262557,-0.578541,-0.219149,-0.47813,0.405550,-0.983740,-0.101547,0.591265,-0.248769,0.047957,-1.583749,0.300263,-1.522081,-0.261454,0.053490,-0.066654,-0.066413,0.054726,-0.093253,-0.093253,0.033930,0.032703,0.033930,-1.272487,-0.406335,-0.156837,-0.805371,0.720036,-0.357419,165000.0
4,1472,0.941011,-0.262557,2.315631,-0.219149,-0.47813,0.405550,-1.397087,-0.622532,-0.897982,-1.183389,0.881607,0.139318,-0.326315,-0.018026,-0.332922,-0.298382,-0.256260,-0.256163,-0.244815,-0.193476,-0.193476,-0.369229,-0.368686,-0.369229,0.744710,-0.406335,-0.156837,-0.805371,0.720036,-0.536427,127000.0


## Build two Models
We have to types of homes:
1. The home has a previous Sale Price.
2. The home does not have a previous Sale Price.

We believe that we can get better results if we build two models on these two datasets seperately.

In [4]:
# Building Model for Old Homes.
data = data[data['PREVSALEPRICE'] != np.nan]

print("Data Shape: ", data.shape)

Data Shape:  (71672, 32)


## Train/Validation/Test Split

We will split our dataset into three sets:
* Training - (77%)
* Validation - (16.5%)
* Testing - (16.5%)

In [5]:
from sklearn.model_selection import train_test_split

X = data.drop(['Unnamed:0', 'SALEPRICE'], axis=1)
Y = data['SALEPRICE']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle=True, random_state=100) # Reproducible
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=100) # Reproducible

print("Training Shape: ", X_train.shape)
print("Validation Shape: ", X_val.shape)
print("Testing Shape: ", X_test.shape)

Training Shape:  (48020, 30)
Validation Shape:  (11826, 30)
Testing Shape:  (11826, 30)


In [6]:
X_train.head(1)

,STYLE,ROOF,EXTFINISH,BASEMENT,ZIPCODE,HOOD,YEARSOLD,PREVSALEPRICE,PREVSALEDATE,SALEDATE,COUNTS,CDU,GRADE,CONDITION,LOTAREA,COUNTYTOTAL,FAIRMARKETTOTAL,LOCALTOTAL,COUNTYBUILDING,FAIRMARKETBUILDING,LOCALBUILDING,COUNTYLAND,FAIRMARKETLAND,LOCALLAND,STORIES,TOTALROOMS,BEDROOMS,FULLBATHS,HALFBATHS,FINISHEDLIVINGAREA
49444,-0.959051,-0.262557,-0.578541,-0.219149,-0.22669,0.066582,-0.053707,-0.62254,2.349558,0.346065,1.679529,0.139318,0.926842,-0.018026,-0.010741,-0.462293,-0.418527,-0.418555,-0.566382,-0.511574,-0.511574,-0.015235,-0.016247,-0.015235,-1.272487,-1.013972,-1.343458,-0.805371,-1.013469,-1.042954


# Modeling
We will build some models on our data and see which model performs the best. The following models will be evaluated:

* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

## Evaluation

We are choosing to evaluate our performance using the `Mean Squared Logarithmic Error` instead of more traditional methods (such as RMSE). This is because the MSLE is more **robust against outliers**, and it **penalizes under-estimation more than over-estimations**, which should be the preference to someone hoping to maximize their profits off of their home.

Home Sales are skewed to the right, with a few really expensive homes. The MSLE will help protect against those outliers, and help us determine which model and parameters to use.

We are using the MSLE metric for the following:
* Model Selection
* Hyper Parameter Tuning
* Testing

We then compare our results against Zillow using their customer facing metrics. There is more on this later in the Notebook.

References:
* https://www.kaggle.com/c/zillow-prize-1/discussion/33899
* https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/loss-functions/mean-squared-logarithmic-error-(msle)

## Testing Function
Here we will define a function that accepts a model, parameters, and data. This model will build the model and test it.

This function will be useful for testing all of our models.

In [7]:
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time

def test_model(model, params, model_name, X_train, y_train, X_test, Y_test):
    """Trains model, and evaluates it.
        PARAMS:
            model_choice - SKLearn Model: Model to be trained
            params - dictionary: Dictionary of parameters to feed the model
            X_train - DataFrame: Training Data, Features
            y_train - DataFrame: Training Data, Targets
            X_test - DataFrame: Testing Data, Features
            y_test - DataFrame: Testing Data, Targets
        
        RETURNS:
            Mean Squared Log Error, Tree Explainer, Model Name - tuple(float, Explainer, String): Accuracy for specified model and parameters. 
    """
    print("Begin ", model_name)
    start = time.time()

    # Run model and get predictions and Mean Squared Log Error
    clf = model(**params)
    clf.fit(X_train, y_train)
    
    predictions = clf.predict(X_test)
    if predictions.min() < 10000:
        print("\t- [WARNING] - Out of bounds prediction... Replacing with 10,000")
        predictions[predictions < 0] = 10000
    
    msle = mean_squared_log_error(Y_test, predictions)
    rmse = mean_squared_error(Y_test, predictions)**0.5
    r2 = r2_score(Y_test, predictions)
    
    print(f"\t- MSLE: ", msle)
    print(f"\t- RMSE: ", rmse)
    print(f"\t- R^2: ", r2)
    
    print("\t- Time Elapsed: ", time.time() - start)
    
    return (msle, clf, model_name)

## Testing Models
We will run all of the following models through the testing function:
* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

Each of these models will be trained on the `Training Set` and evaluated on the `Validation Set`. The model that performs the best on the evaluation set (lowest `Mean Squared Log Error`), will be promoted to the next phase of model tuning (Hyper Parameter Tuning).

In [8]:
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

knc_params = {}
svr_params = {}
gbr_params = {}
xgb_params = {'objective':'reg:squarederror'}
rf_params = {}

models_to_test = [
    (xgb.XGBRegressor, xgb_params, "XGBoost"),
    (RandomForestRegressor, rf_params, "Random Forest"),
    (KNeighborsRegressor, knc_params, "KNeighbors"),
    (LinearSVR, svr_params, "Support Vector Machine"),
    (GradientBoostingRegressor, gbr_params, "GradientBoostingRegressor")
]

results = []
for model in models_to_test:
    results += [test_model(model[0], model[1], model[2], X_train, y_train, X_val, y_val)]

Begin  XGBoost
	- MSLE:  0.05181768507893286
	- RMSE:  30195.035113832644
	- R^2:  0.9366202693432062
	- Time Elapsed:  3.4366214275360107
Begin  Random Forest
	- MSLE:  0.0464578237919611
	- RMSE:  29557.171314802592
	- R^2:  0.9392697528065572
	- Time Elapsed:  87.45868921279907
Begin  KNeighbors
	- MSLE:  0.07399146497433481
	- RMSE:  38598.279014334155
	- R^2:  0.896434479162101
	- Time Elapsed:  42.885947942733765
Begin  Support Vector Machine
	- [WARNING] - Out of bounds prediction... Replacing with 10,000
	- MSLE:  1.1458760973104432
	- RMSE:  132786.8805997945
	- R^2:  -0.2257154168580282
	- Time Elapsed:  0.14203405380249023
Begin  GradientBoostingRegressor
	- MSLE:  0.05272694400627664
	- RMSE:  30522.76732652569
	- R^2:  0.9352369753656048
	- Time Elapsed:  24.06377649307251


In [12]:
# Sort results by the best accuracy.
results.sort(key = lambda x: x[0], reverse=False)

print("Best Model: ", results[0][2])

Best Model:  Random Forest


## Hyper Parameter Tuning
`Random Forest` has the best performance.

We will now run a Grid Search on it to see which parameters might be most effective. We have already run a Randomized Grid Search (in the past) to see how we should narrow down our parameter space. 

    > NOTE: We are choosing to use a smaller `max_depth`. We do this so it can act as a regularization term. We may not get better results (than a higher max_depth), but we will get a more general model.

In [13]:
from sklearn.model_selection import GridSearchCV

def grid_search(clf, params, model_name, X_train, y_train, X_test, y_test):
    """Run Grid Search on a model"""
    print("Begin ", model_name)
    
    start = time.time()

    clf = GridSearchCV(clf(), params, refit=False, scoring='neg_mean_squared_log_error', cv=3, n_jobs=-1, verbose=10)
    clf.fit(X_train, y_train)
    
    print("\t- Best Score: ", clf.best_score_)
    print("\t- Time Elapsed: ", time.time() - start)
    
    return (clf.best_score_, clf, model_name)

# Fitting 3 folds for each of 144 candidates, totalling 432 fits
params_rf = {
    'bootstrap': [True], # Whether to bootstrap samples when building tress.
    'max_depth': [10,20,30,40,50], # Max depth of tree. NOTE: Larger number increases risk for overfitting.
    'max_features': ['auto'], # Num features to consider when looking for the best split.
    'min_samples_leaf': [3,4], # Minimum num of samples required to be at a leaf node.
    'min_samples_split': [4,5,6], # Minimum num of samples required to split an internal node
    'n_estimators': [700,800] # The number of trees in the forest
}

models_to_tune = [
    (RandomForestRegressor, params_rf, "RandomForest")
]

tuned_results = []
for model in models_to_tune:
    tuned_results += [grid_search(model[0], model[1], model[2], X_train, y_train, X_val, y_val)]

Begin  RandomForest
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 26.3min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed: 30.0min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 39.3min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 45.9min
[Parallel(n_jobs=-1)]: Done  96 out of 108 | elapsed: 59.0min remaining:  7.4min


	- Best Score:  -0.05283532946959762
	- Time Elapsed:  3875.348260641098


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 64.6min finished


## Best Parameters
We can view the best parameters by extracting `cv_results_`. This will allow us to see how the best parameters compared to the other parameters.

During the Hyper Parameter Tuning process, this was useful in order to narrow the search space down. For example, We saw consistently worse results during the following:
* bootstrap : False
* max_features : sq_rt

Armed with this knowledge, we were able to reduce the number of parameters we need to search over, so GridSearch might be more efficient

In [14]:
# Grab parameter results
param_results = pd.DataFrame(tuned_results[0][1].cv_results_).sort_values('rank_test_score')

# Write Parameters to CSV
import os
# if file does not exist write header 
if not os.path.isfile('random_forest_params.csv'):
    param_results.to_csv('random_forest_params.csv', header=True)
else: # else it exists so append without writing the header
    param_results.to_csv('random_forest_params.csv', mode='a', header=False)

param_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
13,454.233283,1.029309,3.155558,0.070244,True,50,auto,3,4,800,"{'bootstrap': True, 'max_depth': 50, 'max_feat...",-0.052259,-0.052335,-0.053913,-0.052835,0.000763,1
25,457.191202,1.877035,3.149411,0.045643,True,60,auto,3,4,800,"{'bootstrap': True, 'max_depth': 60, 'max_feat...",-0.052275,-0.052346,-0.053895,-0.052839,0.000748,2
5,455.799587,1.942359,3.147580,0.021062,True,40,auto,3,6,800,"{'bootstrap': True, 'max_depth': 40, 'max_feat...",-0.052298,-0.052350,-0.053923,-0.052857,0.000754,3
26,401.076492,3.206994,2.743830,0.027984,True,60,auto,3,5,700,"{'bootstrap': True, 'max_depth': 60, 'max_feat...",-0.052296,-0.052389,-0.053904,-0.052863,0.000737,4
16,396.036934,1.896560,2.765608,0.036264,True,50,auto,3,6,700,"{'bootstrap': True, 'max_depth': 50, 'max_feat...",-0.052249,-0.052409,-0.053938,-0.052865,0.000761,5


## Evaluate Model

We will evaluate the performance of the `RandomForestRegressor` model on the 
* Training Set
* Validation Set
* Testing Set

Understanding how the results varies within each of the data sets will help us understand how well the model is generalizing to new data. We expect to see a similar results across all data sets.

> NOTE: We will only using the results on the `testing` for determining to accept or decline our model. 

In [15]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import time

start = time.time()
best_clf = RandomForestRegressor(**tuned_results[0][1].best_params_).fit(X_train, y_train)

print("Time Elapsed: ", time.time() - start)

predictions = best_clf.predict(X_train)
msle = mean_squared_log_error(y_train, predictions)
rmse = mean_squared_error(y_train, predictions)**0.5
r2 = r2_score(y_train, predictions)

print("TRAINING")
print("\tMean Squared Log Error: ", msle)
print("\tRoot Mean Squared Error: ", rmse)
print("\tR2-Score: ", r2)

predictions = best_clf.predict(X_val)
msle = mean_squared_log_error(y_val, predictions)
rmse = mean_squared_error(y_val, predictions)**0.5
r2 = r2_score(y_val, predictions)

print("VALIDATION")
print("\tMean Squared Log Error: ", msle)
print("\tRoot Mean Squared Error: ", rmse)
print("\tR2-Score: ", r2)

test_predictions = best_clf.predict(X_test)
msle = mean_squared_log_error(y_test, test_predictions)
rmse = mean_squared_error(y_test, test_predictions)**0.5
r2 = r2_score(y_test, test_predictions)

print("TESTING")
print("\tMean Squared Log Error: ", msle)
print("\tRoot Mean Squared Error: ", rmse)
print("\tR2-Score: ", r2)

Time Elapsed:  448.40923166275024
TRAINING
	Mean Squared Log Error:  0.018423922193051372
	Root Mean Squared Error:  15672.10244968276
	R2-Score:  0.9828514640351864
VALIDATION
	Mean Squared Log Error:  0.047043294994783304
	Root Mean Squared Error:  27434.759951099644
	R2-Score:  0.9474669522446908
TESTING
	Mean Squared Log Error:  0.052649577636754874
	Root Mean Squared Error:  29443.71743270773
	R2-Score:  0.9408978148920639


## Zillow's Zestimate
Zillow presents their results via a table that displays the following:
* Median Error
* Homes With ZESTIMATES
* Within 5% of Sale Price
* Within 10% of Sale Price
* Within 20% of Sale Price

We will develop the same metrics from our results and compare them to Zillows results on `Pittsburgh PA`

In [16]:
y_test = y_test.reset_index(drop=True)

# Concatentate Predictions with Actual
comparison = pd.concat([pd.Series(test_predictions), y_test], axis=1)
comparison = comparison.rename(columns={0:'PREDICTION'})

# Calculate `Percent Error`
## |(Prediction - SalePrice) / SalePrice|
comparison['PERCENT ERROR'] = abs((comparison['PREDICTION'] - comparison['SALEPRICE']) / comparison['SALEPRICE'])

# Write to CSV
comparison.to_csv('predictions.csv')

In [17]:
comparison.head()

,PREDICTION,SALEPRICE,PERCENT ERROR
0,97497.854903,97000.0,0.005133
1,133042.535299,125500.0,0.060100
2,103397.784652,155000.0,0.332918
3,137807.790798,140500.0,0.019162
4,34634.551918,36500.0,0.051108


In [18]:
# Obtain Metrics to create table.

area = 'Pittsburgh, PA'
median_error = comparison['PERCENT ERROR'].median() * 100
total_estimates = comparison.shape[0]
five_percent = comparison[comparison['PERCENT ERROR'] <= 0.05].shape[0] / comparison.shape[0] * 100
ten_percent = comparison[comparison['PERCENT ERROR'] <= 0.10].shape[0] / comparison.shape[0] * 100
twenty_percent = comparison[comparison['PERCENT ERROR'] <= 0.20].shape[0] / comparison.shape[0] * 100

column_names = ['METROPOLITAN AREA', 'MEDIAN ERROR', 'HOMES WITH ESTIMATE', 'WITHIN 5% OF SALEPRICE', 'WITHIN 10% OF SALEPRICE', 'WITHIN 20% OF SALEPRICE']

results_lst = [area, median_error, total_estimates, five_percent, ten_percent, twenty_percent]
zillow_lst = ['Pittsburgh, PA', 2.5, 10900, 77.6, 93.2, 98.4]

results = pd.DataFrame([results_lst, zillow_lst], index=['Our Results', 'Zillow'], columns=column_names)

results.head()

,METROPOLITAN AREA,MEDIAN ERROR,HOMES WITH ESTIMATE,WITHIN 5% OF SALEPRICE,WITHIN 10% OF SALEPRICE,WITHIN 20% OF SALEPRICE
Our Results,"Pittsburgh, PA",8.19912,13628,34.098914,56.904902,79.490754
Zillow,"Pittsburgh, PA",2.50000,10900,77.600000,93.200000,98.400000


# Feature Importance:

### Random Forest Feature Importance
Random Forest has a built in feature importance, which will tell us how important each feature is. However, this does not tell us what kind of values the features should have.

In [19]:
features = X.columns.tolist()
importances = best_clf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

## Results

We tested the following models:
* XGBoost
* RandomForest
* KNearest Neighbors
* Support Vector Machine
* Gradient Boosted Decision Tree

The best performing model was `RandomForest`. Once we found the best performing model, we implemented a `Grid Search` which aimed to exhaustively explore a specified grid and find the best parameters. Once we tuned our parameters, we tested our predictions on our Training, Validation, and Testing set.

Our testing set had a RMSE of `4780.84`. This indicates that we still have a lot of room for improvements, so we might want to attempt a different approach.